In [1]:
import sys
sys.path.append('../../')
import os

import pandas as pd
from utils import fill_empty, VColumns
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

# Load input file

In [2]:
df1 = pd.read_csv("v1-start.csv")

# Start elaborating columns

Create a new dataframe with empty values, so that we can start filling in values from the input dataframe, without overriding columns.

In [4]:
df1.head()

,id,ID personalizzato,name,winery,size,vintage,sales_eur,SOTTERRAN,SPECCHI,SCAFFALE ALTO,MARGINI NASCOSTI,WEINLAGER,L,qty,purchase_price_eur
0,NaN,NaN,LE VIGNE BAROLO,LUCIANO SANDRONE,BORDEAUX_JEROBOAM,2019,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,200.0
1,NaN,NaN,FALLETTO LE ROCCHE,BRUNO GIACOSA,NaN,2019,NaN,6.0,NaN,NaN,NaN,NaN,NaN,6.0,199.0
2,NaN,NaN,BAROLO VIA NUOVA,CHIARA BOSCHIS,NaN,2020,NaN,6.0,5.0,NaN,NaN,NaN,NaN,11.0,76.0
3,NaN,NaN,BAROLO SERRADENARI,RENATO RATTI,NaN,2021,NaN,12.0,NaN,NaN,NaN,NaN,NaN,12.0,65.0
4,NaN,NaN,LE PERGOLE TORTE,MONTEVERTINE,JEROBOAM,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,300.0


In [5]:
# create new empty dataframe
df = pd.DataFrame(columns=VColumns.v2())
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible


In [6]:
import re

# copy values from original columns to new columns
df['external_id'] = None
df['type'] = None
df['name'] = df1['name']
df['winery_name'] = df1['winery']
# First extract vintage from name
df["vintage"] = df1["vintage"]
df["quantity"] = df1["qty"]
df["internal_notes"] = None
df["visible"] = True

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,NaN,NaN,LE VIGNE BAROLO,LUCIANO SANDRONE,NaN,NaN,2019,NaN,NaN,1.0,NaN,None,True
1,NaN,NaN,FALLETTO LE ROCCHE,BRUNO GIACOSA,NaN,NaN,2019,NaN,NaN,6.0,NaN,None,True
2,NaN,NaN,BAROLO VIA NUOVA,CHIARA BOSCHIS,NaN,NaN,2020,NaN,NaN,11.0,NaN,None,True
3,NaN,NaN,BAROLO SERRADENARI,RENATO RATTI,NaN,NaN,2021,NaN,NaN,12.0,NaN,None,True
4,NaN,NaN,LE PERGOLE TORTE,MONTEVERTINE,NaN,NaN,2019,NaN,NaN,0.0,NaN,None,True


In [223]:
# size
df['size'] = df1["size"].fillna("BOTTLE")

# fill empty prices with 0
df1['price_eur'] = df1['sales_eur'].fillna(0)
df1['purchase_price_eur'] = df1['purchase_eur'].fillna(0)

# convert prices to cents
df['price'] = df1['price_eur'].apply(lambda x: int(float(str(x).replace('€', ''))*100)) # convert to cents
df['purchase_price'] = df1['purchase_price_eur'].apply(lambda x: int(float(str(x).replace('€', ''))*100)) # convert to cents

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,NaN,NaN,Metodo Charmat Ruio Brut,Malibran,NaN,BOTTLE,0,3500,585,276,NaN,NaN,True
1,NaN,NaN,Metodo Charmat Rose Brut,Malibran,NaN,BOTTLE,0,4000,585,58,NaN,NaN,True
2,NaN,NaN,Metodo Charmat Col Fondo Sottoriva,Malibran,NaN,BOTTLE,0,3500,585,14,NaN,NaN,True
3,NaN,NaN,Metodo Charmat Credamora,Malibran,NaN,BOTTLE,2021,4500,700,7,NaN,NaN,True
4,NaN,NaN,Metodo Charmat Col Fondo vintage,Malibran,NaN,BOTTLE,2016,0,0,3,NaN,NaN,False


In [224]:
df = fill_empty(df, VColumns.v2(), False)
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,,,Metodo Charmat Ruio Brut,Malibran,,BOTTLE,0,3500,585,276,,,True
1,,,Metodo Charmat Rose Brut,Malibran,,BOTTLE,0,4000,585,58,,,True
2,,,Metodo Charmat Col Fondo Sottoriva,Malibran,,BOTTLE,0,3500,585,14,,,True
3,,,Metodo Charmat Credamora,Malibran,,BOTTLE,2021,4500,700,7,,,True
4,,,Metodo Charmat Col Fondo vintage,Malibran,,BOTTLE,2016,0,0,3,,,False


# Merge files and write output

If there are multiple files, merge them into one and create single output file.

In [225]:
df.head()

df_out = pd.concat([df], ignore_index=True)
df_out.to_csv("v2-cleaned.csv", index=False)

bool()

False